In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from keras_tuner import RandomSearch

In [2]:
Trails=3 # Number of models to train
epochs=2 # number of epoch per model

# Get the MNIST dataset

In [3]:
(xtrain,ytrain),(xval,yval)=mnist.load_data()
xtrain=np.expand_dims(xtrain.astype('float32')/255.,-1)
xval=np.expand_dims(xval.astype('float32')/255.,-1)
ytrain=to_categorical(ytrain,10)
yval=to_categorical(yval,10)

In [4]:
def build_model(hp):
    """
    Function to build a TF model based on hyperparameter values.

    Args:
    hp (hyperparameter): values

    Returns:
    Model: compiled model
    """
    num_layers = hp.Int('num_layers', 2, 8, default=6)
    lr = hp.Choice('learning_rate', [0.0001, 0.05])
    inputs = layers.Input(shape=(28, 28, 1))
    x = inputs

    for idx in range(num_layers):
        idx_str = str(idx)
        filters = hp.Int('filters_' + idx_str, 32, 256, step=32, default=64)
        x = layers.Conv2D(filters=filters, kernel_size=3, padding='same', activation='relu')(x)

        if x.shape[1] >= 8:
            pool_type = hp.Choice('pool_' + idx_str, values=['max', 'avg'])
            if pool_type == 'max':
                x = layers.MaxPooling2D(2)(x)
            elif pool_type == 'avg':
                x = layers.AveragePooling2D(2)(x)

    x = layers.Flatten()(x)
    outputs = layers.Dense(10, activation='softmax')(x)

    # Build model
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=Adam(lr), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [5]:
tuner=RandomSearch(build_model,objective='val_accuracy',max_trials=Trails,project_name='MNIST_results')

Reloading Tuner from .\MNIST_results\tuner0.json


In [6]:
tuner.search_space_summary()

Search space summary
Default search space size: 12
num_layers (Int)
{'default': 6, 'conditions': [], 'min_value': 2, 'max_value': 8, 'step': 1, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.0001, 'conditions': [], 'values': [0.0001, 0.05], 'ordered': True}
filters_0 (Int)
{'default': 64, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
pool_0 (Choice)
{'default': 'max', 'conditions': [], 'values': ['max', 'avg'], 'ordered': False}
filters_1 (Int)
{'default': 64, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
pool_1 (Choice)
{'default': 'max', 'conditions': [], 'values': ['max', 'avg'], 'ordered': False}
filters_2 (Int)
{'default': 64, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
filters_3 (Int)
{'default': 64, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
filters_4 (Int)
{'default': 64, 'conditions': [], 'min_value

In [7]:
tuner.search_space_summary

<bound method BaseTuner.search_space_summary of <keras_tuner.src.tuners.randomsearch.RandomSearch object at 0x0000018E6480C790>>

In [8]:
tuner.search(xtrain,ytrain,batch_size=128,epochs=2,validation_data=(xval,yval))

Trial 3 Complete [00h 17m 40s]
val_accuracy: 0.11349999904632568

Best val_accuracy So Far: 0.9728999733924866
Total elapsed time: 00h 32m 54s


In [9]:
tuner.results_summary()

Results summary
Results in .\MNIST_results
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
num_layers: 8
learning_rate: 0.0001
filters_0: 32
pool_0: avg
filters_1: 192
pool_1: max
filters_2: 64
filters_3: 96
filters_4: 64
filters_5: 32
filters_6: 64
filters_7: 64
Score: 0.9728999733924866

Trial 1 summary
Hyperparameters:
num_layers: 5
learning_rate: 0.0001
filters_0: 32
pool_0: avg
filters_1: 128
pool_1: avg
filters_2: 224
filters_3: 64
filters_4: 32
filters_5: 224
filters_6: 256
filters_7: 128
Score: 0.9587000012397766

Trial 2 summary
Hyperparameters:
num_layers: 5
learning_rate: 0.05
filters_0: 224
pool_0: max
filters_1: 160
pool_1: max
filters_2: 96
filters_3: 64
filters_4: 192
filters_5: 32
filters_6: 256
filters_7: 96
Score: 0.11349999904632568


In [10]:
best_model=tuner.get_best_models(num_models=1)[0]
best_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 average_pooling2d (Average  (None, 14, 14, 32)        0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 192)       55488     
                                                                 
 max_pooling2d (MaxPooling2  (None, 7, 7, 192)         0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 64)          110656